In [26]:
# Skye Platts, 33775116
# Pitch shift wav file

import numpy as np
import math

sound = wavReadMono("song1.wav")

fftSize = 1024
hopSize = 128
windowIndex = 0

outputBuffer = [0] * np.size(sound)
windowFunction = np.hanning(fftSize)
lastInputPhases = [0] * fftSize
lastOutputPhases = [0] * fftSize
analysisMagnitudes = [0] * (fftSize//2 + 1)
analysisFrequencies = [0] *(fftSize//2 + 1)
synthesisMagnitudes = [0] * (fftSize//2 + 1)
synthesisFrequencies = [0] * (fftSize//2 + 1)

# Factor for pitch shift
pitchShift = 0.9



while (windowIndex < np.size(sound)):
    
    endIndex = windowIndex + fftSize
    
    # a little configuration for condition np.size(sound) != N * fftSize where N is an intiger
    if ((endIndex) < np.size(sound)):
        currentWindow = sound[windowIndex:(windowIndex+fftSize)]
    else:
        currentWindow = sound[windowIndex:np.size(sound)]
        currentWindow = np.concatenate([currentWindow, np.zeros(fftSize-np.size(currentWindow))]) 
        
    # Window time domain input
    currentWindow = np.array(currentWindow*windowFunction)
    
    # FFT
    spectrum = np.fft.fft(currentWindow)
    
    # Analysis
    for n in range(fftSize//2):
        
        magnitude = 2 / fftSize * np.abs(spectrum[n])
        phase = np.angle(spectrum[n])
        
        # Phase difference between last and current hop
        phaseDiff = phase - lastInputPhases[n]
        
        # Subtract the expected phase difference for the centre frequency
        # Return true phase difference
        binCentreFrequency = 2 * pi * n / fftSize
        phaseDiff = phaseDiff - binCentreFrequency * hopSize
        if (phaseDiff >= 0):
            phaseDiff = math.fmod(phaseDiff + pi, 2 * pi) - pi
        else:
            phaseDiff = math.fmod(phaseDiff - pi, -2 * pi) + pi
        
        # Calculate difference from the centre frequency as a fraction of the number of bins
        binDeviation = phaseDiff * fftSize / hopSize / (2 * pi)
        
        # Add original bin index to get the fraction representation of the bin where the energy is
        analysisFrequencies[n] = n + binDeviation
        
        analysisMagnitudes[n] = magnitude
        
        lastInputPhases[n] = phase
        
        
    for n in range(fftSize//2):
        synthesisMagnitudes[n] = 0
        synthesisFrequencies[n] = 0
    
    # Prepare complex array
    outComplex = 1j * np.arange(fftSize)
        
    # Resynthesis
    for n in range(fftSize//2):
        # Calculate the bin index to store the shifted energy
        newBin = math.floor(n * pitchShift + 0.5)
        if (newBin <= fftSize//2):
            synthesisMagnitudes[newBin] += analysisMagnitudes[n]
            synthesisFrequencies[newBin] = analysisFrequencies[n] * pitchShift
    
    for n in range(fftSize//2):
        magnitude = synthesisMagnitudes[n]
        
        # Calculate offset from bin centre frequency as a fraction of the number of bins
        binDeviation = synthesisFrequencies[n] - n
        
        # Convert to phase value in radians
        phaseDiff = binDeviation * 2 * pi * hopSize / fftSize
        
        # Add back expectted phase difference for the bin calculated from centre frequency
        binCentreFrequency = 2 * pi * n / fftSize
        phaseDiff += binCentreFrequency * hopSize
        
        # Increment the running phase
        outPhase = lastOutputPhases[n] + phaseDiff
        if (outPhase >= 0):
            outPhase = math.fmod(outPhase + pi, 2 * pi) - pi
        else:
            outPhase = math.fmod(outPhase - pi, -2 * pi) + pi
        
        # Calculate complex components from magnitude and phase
        outComplex.real[n] = magnitude * math.cos(outPhase)
        outComplex.imag[n] = magnitude * math.sin(outPhase)
        
        # Copy conjugate into bins above nyquist
        if (n > 0 and n < fftSize//2):
            outComplex.real[fftSize-n] = outComplex.real[n]
            outComplex.imag[fftSize-n] = -outComplex.imag[n]
        
        # Remember last hop's phases
        lastOutputPhases[n] = outPhase
            
        
    # IFFT
    timeDomain = np.fft.ifft(outComplex)
    
    # Window time domain output
    timeDomain *= windowFunction
    
    # Scale by overlap
    timeDomain *= (hopSize/fftSize)
    
    # Add-overlap to output buffer
    if ((windowIndex + fftSize) < np.size(sound)):   
        outputBuffer[windowIndex:windowIndex+fftSize] += timeDomain
    else:
        outputBuffer[windowIndex:np.size(sound)] += timeDomain[np.size(sound)-windowIndex]
    
    windowIndex = windowIndex + hopSize


    
# play takes array
outputBuffer = np.array(outputBuffer)

play(outputBuffer)

/Users/skyeplatts/Documents/PMCLabFiles/PMClibs.py:39: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp  = tmp1.astype(np.int16)
